# Lab 6

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

In [1]:
class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit
        self.value = self.card_value()
    
    def card_value(self):
        if self.rank in ['J', 'Q', 'K']:
            return 10
        elif self.rank=='A':
            return 11
        else:
            return int(self.rank)
    
    def __str__(self):
        return f"{self.rank},{self.suit}"

In [2]:
import random
class Deck:
    def __init__(self, number_decks):
        self.number_decks = number_decks
        self.cards = self.generate_card()
        self.shuffle()
    
    def generate_card(self):
        suits = ['H', 'D', 'C', 'S']
        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        return [Card(rank, suit) for suit in suits for rank in ranks] * self.number_decks
    
    def shuffle(self):
        random.shuffle(self.cards)
    
    def draw_card(self):
        card_draw = self.cards.pop(0)
        return card_draw




In [3]:
deck = Deck(1)
for card in deck.cards:
    print(f"{card}: {card.card_value()}")



Q,D: 10
4,S: 4
10,D: 10
A,D: 11
2,H: 2
10,C: 10
Q,C: 10
5,H: 5
6,S: 6
A,H: 11
J,C: 10
A,S: 11
3,S: 3
6,C: 6
7,C: 7
J,D: 10
3,C: 3
8,D: 8
5,D: 5
J,S: 10
K,H: 10
10,H: 10
4,H: 4
8,C: 8
K,D: 10
A,C: 11
8,S: 8
3,H: 3
4,D: 4
9,C: 9
2,D: 2
7,D: 7
4,C: 4
10,S: 10
8,H: 8
J,H: 10
9,D: 9
3,D: 3
9,S: 9
2,C: 2
9,H: 9
Q,S: 10
7,H: 7
6,H: 6
Q,H: 10
2,S: 2
K,C: 10
K,S: 10
5,S: 5
7,S: 7
5,C: 5
6,D: 6


2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

In [9]:
class Dealer:
    def __init__(self):
        self.hand = []

    def hand_value(self):
        total = sum(card.card_value() for card in self.hand)
        num_aces = sum(1 for card in self.hand if card.rank == 'A')
        
        while total > 21 and num_aces:
            total -= 10  # Convert an Ace from 11 to 1
            num_aces -= 1

        return total
    
    def play(self, deck):
        if self.hand_value()<17:
            self.hand.append(deck.draw_card())

    


In [10]:
class Player:
    def __init__(self, name, chips=1000):
        self.name = name
        self.hand = []
        self.chips = chips

    def hand_value(self):
        total = sum(card.card_value() for card in self.hand)
        num_aces = sum(1 for card in self.hand if card.rank == 'A')
        
        while total > 21 and num_aces:
            total -= 10  # Convert an Ace from 11 to 1
            num_aces -= 1

        return total
    
    def play(self, deck):
        if self.hand_value()<17:
            self.hand.append(deck.draw_card())
    
    def bet(self, n):
        self.chips = self.chips - 10
        return 10

    def winning(self, n):
        return self.chips+n
    
    def loosing(self, n):
        return self.chips-n
    
    def __str__(self):
        return f"{self.name}: {', '.join(str(card) for card in self.hand)} (Total: {self.hand_value()})"

In [11]:
class Human(Player):
    def __init__(self, name, chips):
        super().__init__(name, chips)
    def play(self, deck):
        if self.hand_value()<16:
            self.hand.append(deck.draw_card())
    def bet(self, n):
        if n > self.chips:
            print("Player does not have enough chips.")
        else:
            self.chips = self.chips - n
        return self.chips


In [12]:
class Game:
    def __init__(self, num_players, human_player = False, num_decks=6):
        self.dealer = Dealer()
        self.deck = Deck(num_decks)
        self.players = [Player(f"Player {i+1}") for i in range(num_players)]
        
        if human_player:
            human_name = input("Enter your name: ")
            self.players.append(Human(human_name))
    
    def play_round(self):
        self.dealer.hand = [self.deck.draw_card(), self.deck.draw_card()]
        for player in self.players:
            player.hand = [self.deck.draw_card(), self.deck.draw_card()]
        
        # Players take turns
        for player in self.players:
            player.play(self.deck)
        
        # Dealer plays
        self.dealer.play(self.deck)

        # Display final hands
        self.display_results()

    def display_results(self):
        print("\nFinal results:")
        print(self.dealer)
        for player in self.players:
            print(player)
            if player.hand_value() > 21:
                print(f"{player.name} busts! Dealer wins.")
            elif self.dealer.hand_value() > 21:
                print(f"Dealer busts! {player.name} wins!")
            elif player.hand_value() > self.dealer.hand_value():
                print(f"{player.name} wins!")
            elif player.hand_value() < self.dealer.hand_value():
                print(f"Dealer wins against {player.name}.")
            else:
                print(f"{player.name} ties with the dealer.")

        
    
        


5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

In [13]:
if __name__ == "__main__":
    num_players = int(input("Enter the number of AI players: "))
    include_human = input("Do you want to play? (yes/no): ").strip().lower() == "yes"

    game = Game(num_players=num_players, human_player = include_human, num_decks=6)
    game.play_round()


Final results:
Player 1: 9,S, K,S (Total: 19)
Player 1 wins!


6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 